<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/Processor_PerformanceTester_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [2]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time

## Mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 10549
drwx------ 2 root root     4096 Aug 16 08:53  Dataset
-rw------- 1 root root    86402 Aug 13 09:16  Image_segmentation-Scar.ipynb
drwx------ 2 root root     4096 Aug 17 05:49  Log
drwx------ 2 root root     4096 Aug 16 08:39  Raw_Dataset
-rw------- 1 root root    20445 Aug 24 11:00  RawDataset_Processor-Scar.ipynb
drwx------ 2 root root     4096 Aug 23 14:47  result
-rw------- 1 root root    39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 10637811 Aug 23 14:31  Unet-Scar.ipynb


# **Grobal variable**

In [5]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

RAW_TRAIN_SET_PATH = "/Raw_Dataset/train"
RAW_TEST_SET_PATH = "/Raw_Dataset/test"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 572
NORM_INPUT_H_SIZE = 572

# **Function**

In [6]:
def resize_image(image):
    image = cv2.resize(image, 
                       dsize=(NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE), 
                       interpolation=cv2.INTER_CUBIC
                       )
    
    return image

def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [7]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# 사용 모델

## UNet (Custom)

**마지막 Output 채널 2 → 1 변경**

<img src = "https://drive.google.com/uc?id=14CzAAaKv5v7pVfvugBRbD1xI4IuhmoyT"  width = 640>


# ***데이터셋 학습 비교 실험***


---


*공통사항*
* 원본 이미지 크기 = 244 * 244
* epoch = 20
* batch = 16
* lr = 1e-3


---


*주의사항*
* 이미지의 사이즈는 정규화 되어야 함


---


*참고사항*
* 딥러닝 CNN 의 대표 모델들이 네트워크 입력 사이즈는 224 * 224


---




## 1

*적용사항*
1. Color space를 이용한 영상처리

```
TRAIN || EPOCH 0020 | BATCH  033 /  037 | LOSS  0.0327
TRAIN || EPOCH 0020 | BATCH  034 /  037 | LOSS  0.0325
TRAIN || EPOCH 0020 | BATCH  035 /  037 | LOSS  0.0323
TRAIN || EPOCH 0020 | BATCH  036 /  037 | LOSS  0.0324
TRAIN || EPOCH 0020 | BATCH  037 /  037 | LOSS  0.0337

VALID || EPOCH 0020 | BATCH  012 /  016 | LOSS  0.0361
VALID || EPOCH 0020 | BATCH  013 /  016 | LOSS  0.0362
VALID || EPOCH 0020 | BATCH  014 /  016 | LOSS  0.0365
VALID || EPOCH 0020 | BATCH  015 /  016 | LOSS  0.0356
VALID || EPOCH 0020 | BATCH  016 /  016 | LOSS  0.0361
```
<img src = "https://drive.google.com/uc?id=1p1WPrwgIFR38YDI9fRKRg68G-eyp1oHC"  width = 400>

### Process train dataset

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 7:3
dataset_size = len(train_files)
train_size = int(dataset_size * 0.7)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    #np.save(os.path.join(train_dir, f'scar_{idx:03d}.npy'), val_image)
    #np.save(os.path.join(train_dir, f'label_{idx:03d}.npy'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)
    #np.save(os.path.join(val_dir, f'scar_{val_idx:03d}.npy'), val_image)
    #np.save(os.path.join(val_dir, f'label_{val_idx:03d}.npy'), label_image)

### Process test dataset

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    #np.save(os.path.join(test_dir, f'scar_{idx:03d}.npy'), val_image)
    #np.save(os.path.join(test_dir, f'label_{idx:03d}.npy'), label_image)

## 2

*적용사항*
1. Color space를 이용한 영상처리
2. 불필요 Black boundary 제거
3. Resize to 572*572 (UNet 아키텍쳐 입력 이미지 사이즈)

### Process train dataset

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 7:3
dataset_size = len(train_files)
train_size = int(dataset_size * 0.7)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)
    #np.save(os.path.join(train_dir, f'scar_{idx:03d}.npy'), val_image)
    #np.save(os.path.join(train_dir, f'label_{idx:03d}.npy'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)
    #np.save(os.path.join(val_dir, f'scar_{val_idx:03d}.npy'), val_image)
    #np.save(os.path.join(val_dir, f'label_{val_idx:03d}.npy'), label_image)

### Process test dataset

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)
    #np.save(os.path.join(test_dir, f'scar_{idx:03d}.npy'), val_image)
    #np.save(os.path.join(test_dir, f'label_{idx:03d}.npy'), label_image)

## 3

*적용사항*
1. Color space를 이용한 영상처리
2. 불필요 Black boundary 제거
3. Bordering(cv2.copymakeborder) to 572*572 (UNet 아키텍쳐 입력 이미지 사이즈)

## 4

*적용사항*
1. Black boundary를 제외한 부분을 sliding window를 이용한 down sampling
2. Label image 포함 비율이 20% 이하인 down sample은 제거
3. Color space를 이용한 영상처리
4. 기존 Label과 매칭

## 5

*적용사항*
1. Black boundary를 제외한 부분을 sliding window를 이용한 down sampling
2. Label image 포함 비율이 20% 이하인 down sample은 제거
3. Color space를 이용한 영상처리
4. Super-resolution
5. 기존 Label과 매칭